In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import time
# import keras.layers as kr
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

from keras.models import Model
from keras.applications.resnet import *
# from keras.preprocessing.image_dataset import *
from keras.preprocessing import image
import random

Using TensorFlow backend.


In [22]:
train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values

In [23]:
#define a random seed
random.seed(10)

#get all the train filename
all_locations_name = list(train_xy['id'])

#get all the filename with unique location using step of 5 as there are 5 images for each location
all_locations_name_step5 = [all_locations_name[x:x+5] for x in range(0, len(all_locations_name), 5)]

#randomly get 600 images from the unique location above
validate_locations_name = random.sample(all_locations_name_step5 ,600)

# randomly get one image in each location
validate_name = [random.choice(validate_location) for validate_location in validate_locations_name]

# get the remaining images as the training data
train_name = [train for train in all_locations_name  if train not in validate_name]

In [24]:
#export validate data to csv
out_validate=[]
for i in range(len(validate_name)):
    coor = train_xy[train_xy['id'] == validate_name[i]]
    out_validate.append([validate_name[i], coor.iloc[0]['x'], coor.iloc[0]['y']])
    
validate_csv = pd.DataFrame(out_validate, columns=['id','x','y'])
validate_csv.to_csv('validate.csv',index=False)

In [25]:
# export train data to csv
out_train=[]
for i in range(len(train_name)):
    coor = train_xy[train_xy['id'] == train_name[i]]
    out_train.append([train_name[i], coor.iloc[0]['x'], coor.iloc[0]['y']])
    
train_csv = pd.DataFrame(out_train, columns=['id','x','y'])
train_csv.to_csv('validate_train.csv',index=False)

In [26]:
# read image for cnn
validate_i=[]
train_i=[]
for validate in validate_name:
    img = image.load_img('./train/' + validate + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    validate_i.append(x)
    
for train in train_name:
    img = image.load_img('./train/' + train + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_i.append(x)
    

In [27]:
# usng ResNet101
base_model = ResNet101(weights = 'imagenet', include_top = True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [28]:
# feature extraction 
train_preds = []
validate_preds = []

for i in tqdm(train_i):
    pred = model.predict(i)
    train_preds.append(pred)

for i in tqdm(validate_i):
    pred = model.predict(i)
    validate_preds.append(pred)



100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [03:22<00:00,  2.97it/s]


In [12]:
# save the feature extraction data
f = open(f'./validate_resnet101.pckl','wb')
pickle.dump(validate_preds,f)
f.close()

f = open(f'./validate_train_resnet101.pckl','wb')
pickle.dump(train_preds,f)
f.close()

In [3]:
validate_train_xy = pd.read_csv('validate_train.csv')
validate_xy =  pd.read_csv('validate.csv')
validate_path = validate_xy['id'].values
train_path = validate_train_xy['id'].values

In [4]:
f = open(f'./validate_resnet101.pckl','rb')
validate_preds = pickle.load(f)
f.close()

f = open(f'./validate_train_resnet101.pckl','rb')
train_preds = pickle.load(f)
f.close()

In [5]:
# compute the euclidean distance to predict the location of test image
best_train = []
for validate in tqdm(validate_preds):
    match = [np.sum((train-validate)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)#[:3]
    row = validate_train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=[f'{i+1}' for i in range(6900)])
df.to_csv('validate_resnet101_matches.csv',index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:56<00:00, 10.64it/s]


In [7]:
# get the mean of top 3 similar images
out=[]
for i in range(len(df)):
    coor = [validate_train_xy[validate_train_xy['id']== df[label][i]] for label in ['1','2','3']]
    result = pd.concat(coor)
    out.append([validate_path[i], np.mean(result['x']), np.mean(result['y'])])
    
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('validate_resnet101v2_top3_out.csv',index=False)

In [8]:
resnet101_predict = pd.read_csv('validate_resnet101v2_top3_out.csv')

In [15]:
# calculate MAE
MAE = np.abs(resnet101_predict ['x']-validate_xy['x']) + np.abs(resnet101_predict ['y']-validate_xy['y'])
MAE = np.sum(MAE)/1200

In [16]:
MAE

9.740722222259722